In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd


In [ ]:
TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
metadata = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
metadata

In [ ]:
# TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
TABLE_PATH = src.config.SILVER / "account"


In [ ]:
account = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
account

In [ ]:
account_cleaned = account[
    (account['mjaccttypcd'].isin(['CML','MTG','MLN']))
].copy()

In [ ]:
account_cleaned

In [ ]:
TABLE_PATH = src.config.SILVER / "property"
property = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
property

In [ ]:
property.info()

In [ ]:
# Acct prop linking
TABLE_PATH = src.config.SILVER / "account_property_link"
acct_prop_link = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
acct_prop_link

In [ ]:
account_cleaned.info()

In [ ]:
acct_prop_link.info()

In [ ]:
merged_df = account_cleaned.merge(acct_prop_link, how='inner', on='acctnbr').merge(property, how='left', on='propnbr')

In [ ]:
TABLE_PATH = src.config.SILVER / "insurance"
insurance = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
insurance

In [ ]:
TABLE_PATH = src.config.SILVER / "account_property_link"
acct_prop_link = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
acct_prop_link